In [52]:
import pandas as pd

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

directory = ''

# Load your Excel file using openpyxl
df1 = pd.read_excel(f'{directory}/Output/Formating Urban/Section_codes_current.xlsx', header=0, engine='openpyxl')

# Loading legal data
df2 = pd.read_csv(f'{directory}/Data/master_table1.csv')

# Creating subset of data (enter lowercase)
desired_state1 = "Nebraska"
desired_state2 = "nebraska"
subset_df1 = df1[df1['STATE'] == desired_state1]
subset_df2 = df2[df2['State'] == desired_state2]

# # Check for NA
# if subset_df2['section'].isna().any():
#     print("The 'section' column contains missing values.")
# else:
#     print("The 'section' column does not contain missing values.")

pattern = r'nebraska_statutes_(.*)'
subset_df2['section1'] = subset_df2['0'].str.extract(pattern)
subset_df2['section1'] = subset_df2['section1'].str.replace('_', '-', case=False)

subset_df2['section'] = subset_df2['section1']

# pattern2 = r'§(.*)'
pattern2 = r'§\s*(\d+-\d+,?\d*);?'

matches = subset_df1['Notes'].str.findall(pattern2)

# Store the matches in a new column as a list
subset_df1['Section Code Re'] = matches

# Explode the list into separate rows
subset_df1 = subset_df1.explode('Section Code Re')


subset_df1 = subset_df1.drop_duplicates(subset=['STATE', 'Regulatory Type Full', 'Section Code', 'Section Code Re'])

# Drop the original 'Section Code Re' column if needed
# subset_df1_exploded.drop(columns=['Section Code Re'], inplace=True)

# Reset the index if necessary
# subset_df1_exploded.reset_index(drop=True, inplace=True)

# subset_df1['Section Code Re'] = subset_df1['Notes'].str.extract(pattern2)
subset_df1['Section Code Re'] = subset_df1['Section Code Re'].str.replace(',', '-', case=False)
subset_df1['Section Code Re'] = subset_df1['Section Code Re'].str.replace(']', '', case=False)
subset_df1['Section Code Re'] = subset_df1['Section Code Re'].str.strip()

# Clean section column
# subset_df2['section'] = subset_df2['section'].str.replace('section-', '', case=False)
# subset_df2['section'] = subset_df2['section'].str.replace('-', ':', 1)  # Replace first hyphen with colon
# subset_df2['section'] = subset_df2['section'].str.upper()
# subset_df2['section'] = subset_df2['section'].apply(lambda x: x[:x.rfind('-')] + '.' + x[x.rfind('-')+1:])

# subset_df2.head(5)

# Merge the subsetted dataframes on 'Section Code' and 'Section'
merged_df = pd.merge(subset_df1, subset_df2[['State','section', 'text', 'section1']], left_on='Section Code Re', right_on='section1', how='left')
# merged_df = merged_df[merged_df['Section Code'].notna()] 

# merged_df.head()

# Save the merged data to an Excel file
merged_df.to_excel(f'{directory}/Output/Matching Urban to Drexel/Section_codes_merged_{desired_state2}.xlsx', index=False)



In [51]:
# subset_df1.head(10)

,Unnamed: 0,Unnamed: 0.1,STATE,Regulatory Type,Regulatory Type Full,Regulatory Body,Regulation Indicator,Notes,Year legislation originally enacted,Year legislation changed,Type of Change,Section Code,Section Code Re
1374,1379,0,Nebraska,STATE,STATE,NaN,Nebraska,Nebraska,NaN,NaN,NaN,NaN,NaN
1375,1380,1,Nebraska,Bifurcated,Bifurcated,NaN,No,Charitable Orgs Not Required to Register,NaN,NaN,NaN,NaN,NaN
1376,1381,2,Nebraska,Registration Office,Registration Office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377,1382,3,Nebraska,Sale of Assets,Sale of Assets - AG,AG,Yes,"Neb. Rev. Stat. § 21-19,126",NaN,NaN,NaN,21-19,21-19-126
1378,1383,4,Nebraska,Sale of Assets,Sale of Assets - Other,Other,No,No,NaN,NaN,NaN,NaN,NaN
1379,1384,5,Nebraska,Mergers,Mergers - AG,AG,Yes,Rel. Org and Public Benefits Corps. Neb. Rev. ...,NaN,NaN,NaN,21-19,21-19-119
1380,1385,6,Nebraska,Mergers,Mergers - Other,Other,Yes,"File with Sec. of State NE ST § 21-19,121; Cou...",NaN,NaN,NaN,21-19,21-19-121
1380,1385,6,Nebraska,Mergers,Mergers - Other,Other,Yes,"File with Sec. of State NE ST § 21-19,121; Cou...",NaN,NaN,NaN,21-19,21-19-119
1382,1387,7,Nebraska,Certificate of Amendments,Certificate of Amendments - AG,AG,No,No,NaN,NaN,NaN,NaN,NaN
1383,1388,8,Nebraska,Certificate of Amendments,Certificate of Amendments - Other,Other,Yes,"Sec. of State [Neb.Rev.St. § 21-19,109]",NaN,NaN,NaN,21-19,21-19-109
